In [1]:
include("./UpperLevel/UpperLevel.jl")
include("utils.jl")

using ArgParse, PrettyTables
using Plots
using .UpperLevel

In [2]:
include("main_upper.jl")
# ATTENTION : CHANGE NB TIME STEPS

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

[ Info: Building model...
[ Info: Built model in 5.54 seconds


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


A JuMP Model
Minimization problem with:
Variables: 206
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 39 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 116 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 174 constraints
`Vector{JuMP.VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 8 constraints
`Vector{JuMP.AffExpr}`-in-`MathOptInterface.RotatedSecondOrderCone`: 12 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 74 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 16 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 17 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Alpha, Beta, DSO_costs, DSO_fixed_costs, DSO_loss_costs, DSO_op_limits, I_sqr_k, P_ij_k, P_sub, Q_ij_k, Q_sub, S_sub, S_sub_capa, User_costs, V_sqr, Y, bilevel, delta_t, nb_sign_days, network_dat

In [ ]:
println(network_topology.sending_lines)
println(network_topology.receiving_lines)


In [ ]:
using JuMP
latex_formulation(upper_model)
# Problem is that etiher P_ij or Q_ij_k


In [ ]:
objective, time = UpperLevel.solve_model(upper_model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")
# 1. What to do is to update the structures when going through this function
# 2. 


In [ ]:
using JuMP, Latexify, PrettyTables

function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

function print_case_description(model::JuMP.AbstractModel)
    network = model[:network_data]
    delta_t = model[:delta_t]
    nb_sign_days = model[:nb_sign_days]
    nb_time_steps = model[:time_steps]  
    bilevel = model[:bilevel]
    N = get_nb_buses(network)
    Ns = get_nb_substations(network)
    buses = network.buses

    energy_consumed = 0
    for i in Ns+1:N
        P_cons = sum(buses[i].load_profile.time_serie .* buses[i].cos_phi)
        energy_consumed += P_cons * delta_t * 365 / nb_sign_days
    end
    
    peak_sub_P = value.(sum(model[:P_sub], dims=2))
    peak_sub_max = maximum(peak_sub_P)
    peak_sub_min = minimum(peak_sub_P)
    case_headers = (["Model", "Objective", "Gap", "Solve Time", "Time periods", "Energy consumed", "Peak sub max", "Peak sub min"], ["", "kEUR/year", "%", "sec.","", "MWh/year", "MW on 1 DT", "MW on 1 DT"])
    case_params = sig_round([(bilevel ? "Bilevel" : "Single Level") objective_value(model) relative_gap(model)*100 solve_time(model) nb_time_steps energy_consumed peak_sub_max peak_sub_min])
    pretty_table(case_params, header=case_headers)
    # returns the table as a Matrix
    return vcat([i[j] for i in case_headers, j in 1:length(case_headers[1])], case_params)
end

function print_network_results(model; latex=false)
    network = model[:network_data]
    nb_lines_built = 0
    for l in network.lines 
        if l.built 
            nb_lines_built += 1
        end
    end

    Ns = get_nb_substations(network)
    buses = network.buses
    nb_substations = 0
    for i in 1:Ns
        if buses[i].built
            nb_substations += 1
        end
    end
 
    lines_cond1 = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, 1]) if isapprox(a, 1, rtol=1e-2)]]
    lines_cond2 = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, 2]) if isapprox(a, 1, rtol=1e-2)]]
    lines_cond3 = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, 3]) if isapprox(a, 1, rtol=1e-2)]]
    lines_cond4 = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, 4]) if isapprox(a, 1, rtol=1e-2)]]
    kpis_header = (["Nb lines built", "Nb substations", "Nb lines cond 1", "Nb lines cond 2", "Nb lines cond 3", "Nb lines cond 4"], ["-", "-", "-", "-", "-", "-"])
    kpis = [nb_lines_built nb_substations lines_cond1 lines_cond2 lines_cond3 lines_cond4]
    [kpis[i] = string(a) for (i, a) in enumerate(kpis) if (i >= 3) && (!isempty(a))]
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
print_network_results(upper_model)

In [ ]:
"""Draws a graph using TikZ."""
function print_network_tikz2(network, time_step, x_scale, y_scale;
                        dir=pwd(), filename="graph", display=true, reshape=false)

    
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}
    \\usepackage{xcolor}
    \\definecolor{Ulg_blue}{RGB}{9, 111, 123}
    \\definecolor{Ulg_red}{RGB}{132, 38, 0}
    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt,  align=center, text width=0.6cm"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt, align=center, text width=0.6cm"
    base_power = network.pu_basis.base_power

    # Creating the .tex file
    file_path = joinpath(dir ,"$filename.tex")
    touch(file_path)

    # Writing the .tex file
    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x_coord = node.coord.x
            y_coord = node.coord.y
            if reshape
                if n in [1, 2]
                    x_coord += 0.5
                elseif n == 16
                    y_coord += 0.1
                elseif n == 20
                    y_coord -=  0.1
                elseif n in [3, 9]
                    if n == 9 
                        x_coord += 0.5  
                    end
                    y_coord -= 1.8
                elseif n in [4, 5]
                    if n == 4
                        x_coord += 0.8
                    else 
                        x_coord += 0.5
                    end
                    y_coord += 0.4
                end
            end
            x = x_scale * x_coord
            y = y_scale * y_coord
            
            ns = get_nb_substations(network)
            nu = n - ns
            if n <= ns
                P_gen = round((b.P_sup[time_step]); digits = 2)
                col = P_gen >= 0 ? "\\color{Ulg_blue}" : "\\color{Ulg_red}"
                if b.built
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$  \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub), very thick];\n")
                else 
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$ \\vspace{0.1em}  \\scriptsize \$\\mathbf{$col $P_gen}\$}, $(style_sub)];\n")
                end
            else 
                P_cons = round(b.load_profile.time_serie[time_step] * b.cos_phi; digits = 2)
                P_gen = isnothing(b.PV_installation) ? 0.0 : b.PV_installation.P[time_step]
                P_gen = round(P_gen; digits=2)
                
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$nu}\$  \\vspace{-0.2em} \\scriptsize  \$ \\color{Ulg_blue} \\mathbf{$P_gen}\$ \\vspace{0.1em} \\scriptsize \$ \\color{Ulg_red} \\mathbf{$P_cons}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                p_s = l.P_send[time_step]
                p_r = l.P_rec[time_step]
                if  p_s >= 0                   
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step]
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step])
                end
              
                if l.conductor.name == "Poppy"
                    write(file,
                    "    $(i) ->[\"$(round(p_s, digits=2)), $(round(p_r, digits=2))\"] $(j);\n")
                elseif l.conductor.name == "Oxlip"
                    write(file,
                    "    $(i) ->[\"$(round(p_s, digits=2)), $(round(p_r, digits=2))\", thick] $(j);\n")
                elseif l.conductor.name == "Daisy"
                    write(file,
                    "    $(i) ->[\"$(round(p, digits=2))\", very thick] $(j);\n")
                elseif l.conductor.name == "Tulip"
                    write(file,
                    "    $(i) ->[\"$(round(p, digits=2))\", ultra thick] $(j);\n")
                end
                print(l.conductor.name)
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    end
end




In [ ]:
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                        dir=pwd(), filename="graph", display=true)

    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt, align=center, text width=0.5cm"
    base_power = network.pu_basis.base_power

    # Creating the .tex file
    file_path = joinpath(dir ,"$filename.tex")
    touch(file_path)

    # Writing the .tex file
    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            ns = get_nb_substations(network)
            nu = n - ns
            if n <= ns
                P_gen = round((b.P_sup[time_step] * base_power); sigdigits = 3)
                if b.built
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$  \\tiny  \$ $P_gen\$}, $(style_sub), thick];\n")
                else 
                    write(file,
                    "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$ \\tiny \$ $P_gen\$}, $(style_sub)];\n")
                end
            else 
                P_cons = round(b.load_profile.time_serie[time_step] * base_power; sigdigits = 3)
                P_gen = isnothing(b.PV_installation) ? 0.0 : b.PV_installation.P[time_step]*base_power 
                P_gen = round(P_gen; sigdigits=3)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$nu}\$  \\newline \\tiny  \$+$P_gen-$P_cons\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, sigdigits=3))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    end
end

In [ ]:
# --- https://tikz.dev/tikz-graphs#sec-19.7 ---
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                   dir=pwd(),
                   filename="graph",
                   display=true)

    # Let's insert some boilerplate styling
    # and necessary preamble/postamble
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    
    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt"
    base_power = network.pu_basis.base_power

    file_path = joinpath(dir ,"$filename.tex")

    touch(file_path)

    open(file_path, "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            if n <= get_nb_substations(network)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$}, $(style_sub)];\n")
            else 
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{L}_{$n}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, digits=3))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end

    # command = `cd $dir` |> `lualatex $filename.tex` |> `open $filename.pdf`
    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
        # run(`cd $dir` |> `lualatex $filename.tex` |> `open $filename.pdf`)
    end
end

In [ ]:
# -- Creation of the result plot --
using Graphs

g = Graph(get_nb_nodes(network_topology))
for l in network.lines 
    e = l.edge
    add_edge!(g, e.from_node.id, e.to_node.id)
end

root_dir = splitdir(@__DIR__)[1]
plots_dir = joinpath(root_dir, "plots")
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 

for t in 1:simulation.nb_time_steps
    print_network_tikz2(network, t, 40, 40; dir=thesis_dir, filename="graph_$(t)")
end

# file_path = joinpath(plots_dir ,"graph1.tex")
# touch(file_path)
# run(`cd $plots_dir open graph1.tex`) 

In [ ]:
function communicate(cmd::Cmd, input)
    inp = Pipe()
    out = Pipe()
    err = Pipe()

    process = run(pipeline(cmd, stdin=inp, stdout=out, stderr=err), wait=false)
    close(out.in)
    close(err.in)

    stdout = @async String(read(out))
    stderr = @async String(read(err))
    write(process, input)
    close(inp)
    wait(process)
    return (
        stdout = fetch(stdout),
        stderr = fetch(stderr),
        code = process.exitcode
    )
end


a, = communicate(`julia --project ./main.jl`, "output.txt"); println(a)


In [ ]:
using Graphs
using GraphViz
G = Graph(3) # graph with 3 vertices

# make a triangle
add_edge!(G, 1, 2)
add_edge!(G, 1, 3)
add_edge!(G, 2, 3)

for e in edges(G)
    print(e)
end
# print(nv(G))
# W=zeros(Float64, nv(G), nv(G))
# print(W[1,1])
# for i in 1:nv(G)
#     for j in 1:nv(G)
#         if has_edge(G, i, j)
#             print(i,j)
#             W[i, j] = 0.1
#         end
#     end
# end 


# open("./graph.tmp.dot", "w") do file
#     write(file, "digraph {\n")

#     for n in vertices(G)  # add nodes
#         write(file,
#               "    $n;\n")
#     end

#     for e in edges(G)  # add edges
#         i=src(e); j=dst(e)
#         write(file,
#               "    $i -> $j;\n")
#     end

#     write(file, "}")
# end

# GraphViz.load("graph.tmp.dot")
# current_path = pwd()
# print(current_path)

# import Base
# Base.run(`dot -V`)
# Base.run(`cd $current_path`)
# Base.run(pipeline(`dot -Tpdf graph.tmp.dot`,
#                stdout="graph.tmp.pdf"))
# Base.run(`open graph.tmp.pdf`)


In [ ]:
import Base
Base.run(`dot -V`)

In [ ]:
import Base

Base.run(`ls`)

In [ ]:
include("./Bilevel/Bilevel.jl")
using .Bilevel